### 1. Import Libraries

In [1]:
import emoji
from underthesea import word_tokenize, sent_tokenize
from viet_text_tools import normalize_diacritics
import re 
import pandas as pd

import string
import os

### 2. Data Loading

In [2]:
file_names = ["may_tinh_bang_reviews.csv","dien_thoai_reviews.csv","laptop_reviews.csv", "dong_ho_thong_minh_reviews.csv"]

dir = 'E:\MY CAREER\Internship Project\EDA\dataset\\tgdd\comment'
dataframes = []
for file in file_names:
    path = os.path.join(dir, file)
    try:
        df = pd.read_csv(path)
        dataframes.append(df)
    except pd.errors.ParserError as e:
        print(f"Error reading file")

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.head()

,Unnamed: 0,item_id,cmt_id,username,rating,comment
0,0,247517,r-54899621,Trương thế tình,5,Rất tốt
1,1,247517,r-54762463,Em Huyền,5,Sài rất tốt ạ mượt mà E sài rất kĩ vs lại mặt ...
2,2,247517,r-54740385,Nguyen Trung Vien,4,Xài ổn giữ pin lâu
3,3,247517,r-54721247,Trần Anh Tuấn,2,Máy dùng rất nóng
4,4,247517,r-54633554,Phạm Thị Ngọc,3,"Máy mới xài, nhưng Zalo lâu lâu bị đứng, k biế..."


In [3]:
combined_df.columns

Index(['Unnamed: 0', 'item_id', 'cmt_id', 'username', 'rating', 'comment'], dtype='object')

In [4]:
combine = combined_df[['item_id', 'cmt_id', 'username', 'rating', 'comment']]
combine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26896 entries, 0 to 26895
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   item_id   26896 non-null  int64 
 1   cmt_id    26896 non-null  object
 2   username  26893 non-null  object
 3   rating    26896 non-null  int64 
 4   comment   21553 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.0+ MB


### 3. Data Cleaning

In [5]:
combine = combine.dropna(subset='comment')
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21553 entries, 0 to 26886
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   item_id   21553 non-null  int64 
 1   cmt_id    21553 non-null  object
 2   username  21550 non-null  object
 3   rating    21553 non-null  int64 
 4   comment   21553 non-null  object
dtypes: int64(2), object(3)
memory usage: 1010.3+ KB


In [6]:
combine = combine.dropna(subset='username')
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21550 entries, 0 to 26886
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   item_id   21550 non-null  int64 
 1   cmt_id    21550 non-null  object
 2   username  21550 non-null  object
 3   rating    21550 non-null  int64 
 4   comment   21550 non-null  object
dtypes: int64(2), object(3)
memory usage: 1010.2+ KB


In [7]:
combine.duplicated(subset='cmt_id').sum()

984

In [8]:
combine.drop_duplicates(subset='cmt_id', inplace=True, keep='last')

In [9]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20566 entries, 0 to 26886
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   item_id   20566 non-null  int64 
 1   cmt_id    20566 non-null  object
 2   username  20566 non-null  object
 3   rating    20566 non-null  int64 
 4   comment   20566 non-null  object
dtypes: int64(2), object(3)
memory usage: 964.0+ KB


In [10]:
filtered_data = combine

def check_emoji(text):
    text_without_emoji = emoji.demojize(text)
    return text != text_without_emoji

filtered_data["has_emoji"] = filtered_data["comment"].apply(check_emoji)
filtered_data = filtered_data[filtered_data['has_emoji'] == False]

filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20113 entries, 0 to 26886
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    20113 non-null  int64 
 1   cmt_id     20113 non-null  object
 2   username   20113 non-null  object
 3   rating     20113 non-null  int64 
 4   comment    20113 non-null  object
 5   has_emoji  20113 non-null  bool  
dtypes: bool(1), int64(2), object(3)
memory usage: 962.4+ KB


In [11]:
with open('teencode.txt',"r", encoding="utf-8") as file:
    teencode_words = [line.strip() for line in file]


def get_lower(text):
        return text.lower().strip()

filtered_data['comment'] = filtered_data["comment"].apply(get_lower)
filtered_data["word_count"] = filtered_data["comment"].apply(lambda x: len(x.split()))
filtered_data["teencode_count"] = filtered_data["comment"].apply(lambda x: sum(1 for word in x.split() if word in teencode_words))

filtered_data

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_15816\2890940823.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['comment'] = filtered_data["comment"].apply(get_lower)
C:\Users\Rauciss\AppData\Local\Temp\ipykernel_15816\2890940823.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["word_count"] = filtered_data["comment"].apply(lambda x: len(x.split()))
C:\Users\Rauciss\AppData\Local\Temp\ipykernel_15816\2890940823.py:10: SettingWithCopyWarning: 
A value is trying to be set on 

,item_id,cmt_id,username,rating,comment,has_emoji,word_count,teencode_count
0,247517,r-54899621,Trương thế tình,5,rất tốt,False,2,0
1,247517,r-54762463,Em Huyền,5,sài rất tốt ạ mượt mà e sài rất kĩ vs lại mặt ...,False,17,2
2,247517,r-54740385,Nguyen Trung Vien,4,xài ổn giữ pin lâu,False,5,0
3,247517,r-54721247,Trần Anh Tuấn,2,máy dùng rất nóng,False,4,0
4,247517,r-54633554,Phạm Thị Ngọc,3,"máy mới xài, nhưng zalo lâu lâu bị đứng, k biế...",False,14,1
...,...,...,...,...,...,...,...,...
26878,286738,r-53220008,MINH,3,máy gặp khá ít lỗi kết nối bluetooth hết pin n...,False,18,0
26879,286738,r-52906708,Binh,1,hàng bán ra thì phải bảo hành cho ngta chứ đằ...,False,15,1
26880,286738,r-52760604,Phạm Nhật Trường,5,"kết nối gần ổn, xa 2m ngưng kết nối",False,9,0
26881,286738,r-52698596,Lê Tấn Thành,4,rất hài lòng với sản phẩm này,False,7,0


In [12]:
filtered_data['teencode_count'].value_counts()

0     17515
1      1742
2       502
3       174
4        87
5        39
6        22
7        16
8         4
10        3
9         3
13        3
15        1
16        1
12        1
Name: teencode_count, dtype: int64

In [13]:
filtered_data = filtered_data[(filtered_data["word_count"] >= 5 )& (filtered_data["word_count"] < 50) & (filtered_data["teencode_count"] < 1)]
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8457 entries, 2 to 26881
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   item_id         8457 non-null   int64 
 1   cmt_id          8457 non-null   object
 2   username        8457 non-null   object
 3   rating          8457 non-null   int64 
 4   comment         8457 non-null   object
 5   has_emoji       8457 non-null   bool  
 6   word_count      8457 non-null   int64 
 7   teencode_count  8457 non-null   int64 
dtypes: bool(1), int64(4), object(3)
memory usage: 536.8+ KB


In [14]:
with open('badwords.txt',"r", encoding="utf-8") as file:
    badwords = [line.strip() for line in file]

filtered_data["badwords_count"] = filtered_data["comment"].apply(lambda x: sum(1 for word in x.split() if word in badwords))
filtered_data['badwords_count'].value_counts()

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_15816\3262184657.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["badwords_count"] = filtered_data["comment"].apply(lambda x: sum(1 for word in x.split() if word in badwords))


0    7749
1     615
2      83
3       9
5       1
Name: badwords_count, dtype: int64

In [15]:
filtered_data = filtered_data[(filtered_data["badwords_count"] < 1)]
filtered_data.head(3)

,item_id,cmt_id,username,rating,comment,has_emoji,word_count,teencode_count,badwords_count
2,247517,r-54740385,Nguyen Trung Vien,4,xài ổn giữ pin lâu,False,5,0,0
9,247517,r-54520613,Anh Vũ,3,màng hình ko được nét . chắc do mình cứ tin tư...,False,23,0,0
12,247517,r-54457734,Huy,5,"máy ngon, dùng mượt , giao hàng nhanh và nhiệt...",False,15,0,0


In [16]:
result = filtered_data.copy()

### 4. Data Exploring

In [17]:

def convert_unicode(text):
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    char1252 = char1252.split('|')
    charutf8 = charutf8.split('|')
    
    dic = {}
    for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dic[x.group()], text
    )

result['comment'] = result['comment'].apply(convert_unicode)

result.head()

,item_id,cmt_id,username,rating,comment,has_emoji,word_count,teencode_count,badwords_count
2,247517,r-54740385,Nguyen Trung Vien,4,xài ổn giữ pin lâu,False,5,0,0
9,247517,r-54520613,Anh Vũ,3,màng hình ko được nét . chắc do mình cứ tin tư...,False,23,0,0
12,247517,r-54457734,Huy,5,"máy ngon, dùng mượt , giao hàng nhanh và nhiệt...",False,15,0,0
17,247517,r-54357774,Thịnh,5,các bạn nhân viên làm việc rất chuyên nghiệp. ...,False,40,0,0
22,247517,r-54128896,Huỳnh Thị Thanh,5,"nhân viên nhiệt tình , thân thiện. quá đỗi là ...",False,12,0,0


In [18]:
vowels_to_ids = {}
vowels_table = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
]

for i in range(len(vowels_table)):
    for j in range(len(vowels_table[i]) - 1):
        vowels_to_ids[vowels_table[i][j]] = (i, j)


def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_indexes = -1
    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_indexes == -1: vowel_indexes = index
            else:
                if index - vowel_indexes != 1: return False
                vowel_indexes = index
    return True


def standardize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars = list(word)
    dau_cau = 0
    vowel_indexes = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x == -1: continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:
            dau_cau = y
            chars[index] = vowels_table[x][0]

        if not qu_or_gi or index != 1:
            vowel_indexes.append(index)

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowels_to_ids.get(chars[1])
                chars[1] = vowels_table[x][dau_cau]
            else:
                x, y = vowels_to_ids.get(chars[2], (-1, -1))
                if x != -1: chars[2] = vowels_table[x][dau_cau]
                else: chars[1] = vowels_table[5][dau_cau] if chars[1] == 'i' else vowels_table[9][dau_cau]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        x, y = vowels_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowels_table[x][dau_cau]
            return ''.join(chars)

    if len(vowel_indexes) == 2:
        if vowel_indexes[-1] == len(chars) - 1:
            x, y = vowels_to_ids[chars[vowel_indexes[0]]]
            chars[vowel_indexes[0]] = vowels_table[x][dau_cau]
        else:
            x, y = vowels_to_ids[chars[vowel_indexes[1]]]
            chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    else:
        x, y = vowels_to_ids[chars[vowel_indexes[1]]]
        chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    return ''.join(chars)


def standardize_sentence_typing(text):
    words = text.lower().split()
    for index, word in enumerate(words):
        cw = word.split('/')
        cw = [standardize_word_typing(w) for w in cw]

        if len(cw) == 3: cw[1] = standardize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_punctuation(text):
    punctuation = '!"#$%&\'()*+-/:;<=>?@[\\]^`{|}~'
    translator = str.maketrans('', '', punctuation)
    return text.translate(translator)

In [19]:

result['comment'] = result['comment'].apply(standardize_sentence_typing)
result.head(3)

,item_id,cmt_id,username,rating,comment,has_emoji,word_count,teencode_count,badwords_count
2,247517,r-54740385,Nguyen Trung Vien,4,xài ổn giữ pin lâu,False,5,0,0
9,247517,r-54520613,Anh Vũ,3,màng hình ko được nét . chắc do mình cứ tin tư...,False,23,0,0
12,247517,r-54457734,Huy,5,"máy ngon, dùng mượt , giao hàng nhanh và nhiệt...",False,15,0,0


In [20]:
result['comment'] = result['comment'].apply(remove_punctuation)
result.head(3)

,item_id,cmt_id,username,rating,comment,has_emoji,word_count,teencode_count,badwords_count
2,247517,r-54740385,Nguyen Trung Vien,4,xài ổn giữ pin lâu,False,5,0,0
9,247517,r-54520613,Anh Vũ,3,màng hình ko được nét . chắc do mình cứ tin tư...,False,23,0,0
12,247517,r-54457734,Huy,5,"máy ngon, dùng mượt , giao hàng nhanh và nhiệt...",False,15,0,0


In [21]:
def remove_extra_dots(text):
    return re.sub(r'\.{2,}', '.', text)

with open('stopwords.txt', 'r', encoding='utf-8') as file:
        stopwords = file.read().splitlines()

def remove_stopword(text):
    words = text.split()
    new_words = [word for word in words if word not in stopwords]
    return ' '.join(new_words)


def tokenize(text):
    return re.sub(r'\._+',' . ',word_tokenize(text, format="text"))

def start_processing(text):
    text = remove_extra_dots(text)
    text = tokenize(text)
    text = remove_stopword(text)
    

In [22]:
result['comment'] = result['comment'].apply(remove_extra_dots)
result.head(3)

,item_id,cmt_id,username,rating,comment,has_emoji,word_count,teencode_count,badwords_count
2,247517,r-54740385,Nguyen Trung Vien,4,xài ổn giữ pin lâu,False,5,0,0
9,247517,r-54520613,Anh Vũ,3,màng hình ko được nét . chắc do mình cứ tin tư...,False,23,0,0
12,247517,r-54457734,Huy,5,"máy ngon, dùng mượt , giao hàng nhanh và nhiệt...",False,15,0,0


In [23]:
result['comment'] = result['comment'].apply(tokenize)
result.head(3)

,item_id,cmt_id,username,rating,comment,has_emoji,word_count,teencode_count,badwords_count
2,247517,r-54740385,Nguyen Trung Vien,4,xài ổn giữ pin lâu,False,5,0,0
9,247517,r-54520613,Anh Vũ,3,màng hình_ko được nét . chắc do mình cứ tin_tư...,False,23,0,0
12,247517,r-54457734,Huy,5,"máy ngon , dùng mượt , giao hàng nhanh và nhiệ...",False,15,0,0


In [24]:
result['comment'] = result['comment'].apply(remove_stopword)
result.head(3)

,item_id,cmt_id,username,rating,comment,has_emoji,word_count,teencode_count,badwords_count
2,247517,r-54740385,Nguyen Trung Vien,4,xài ổn giữ pin lâu,False,5,0,0
9,247517,r-54520613,Anh Vũ,3,màng hình_ko nét . chắc mình tin_tưởng apple m...,False,23,0,0
12,247517,r-54457734,Huy,5,"máy ngon , dùng mượt , giao hàng nhanh nhiệt_t...",False,15,0,0


In [25]:
result = result[~result['comment'].str.match(r'^\d$')]
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7749 entries, 2 to 26881
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   item_id         7749 non-null   int64 
 1   cmt_id          7749 non-null   object
 2   username        7749 non-null   object
 3   rating          7749 non-null   int64 
 4   comment         7749 non-null   object
 5   has_emoji       7749 non-null   bool  
 6   word_count      7749 non-null   int64 
 7   teencode_count  7749 non-null   int64 
 8   badwords_count  7749 non-null   int64 
dtypes: bool(1), int64(5), object(3)
memory usage: 552.4+ KB


In [58]:
result.columns

Index(['item_id', 'cmt_id', 'username', 'rating', 'comment', 'has_emoji',
       'word_count', 'teencode_count', 'badwords_count'],
      dtype='object')

In [26]:
result_df = result[['item_id', 'cmt_id', 'username', 'rating', 'comment']]

result_df.to_csv('cleandata/tgdd_comment.csv')